#### Bibliotecas

In [117]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd

#### Aplicação da GAN

##### Gerador

In [124]:
import torch
import torch.nn as nn

class Gerador(nn.Module):
    """
    O gerador recebe um vetor de entrada (contendo números aleatórios) e mapeia para o espaço de saída desejado, 
    produzindo um registro sintético.

    Parâmetros:
        - tamanho_entrada (int): O tamanho do vetor de entrada, que representa a dimensão do espaço latente.
        - tamanho_saida (int): O tamanho do vetor de saída, que representa a dimensão do registro sintético.
        - min_values (torch.Tensor): O tensor contendo os valores mínimos para cada coluna do dataframe original.
        - max_values (torch.Tensor): O tensor contendo os valores máximos para cada coluna do dataframe original.
        - coluna_indices (int): O índice da coluna que precisa ser ajustado para valores específicos.
        - valores_possiveis (list): A lista de valores possíveis para a coluna específica.
    """
    
    def __init__(self, tamanho_entrada, tamanho_saida, min_values, max_values, coluna_indices, valores_possiveis):
        super(Gerador, self).__init__()
        
        # Definição da arquitetura do modelo
        self.modelo = nn.Sequential(
            nn.Linear(tamanho_entrada, 128), # Primeira camada de entrada (linear)
            nn.ReLU(), # Capacitar não-linearidade
            nn.Linear(128, tamanho_saida), # Segunda camada de saída
            nn.Tanh() # Valores de saída no intervalo de (-1 e 1)
        )
        
        self.min_values = min_values
        self.max_values = max_values
        self.coluna_indices = coluna_indices
        self.valores_possiveis = torch.tensor(valores_possiveis)

    def forward(self, x):
        """
        Essa função passar os dados de entrada através do modelo. Recebe o tensor de entrada "x (torch.Tensor)" e retorna
        o tensor de saída gerada pelo modelo. Em PyTorch, tensor são unidades básicas para trabalhar com redes neurais.

        Parâmetros:
            - x (torch.Tensor): O tensor de entrada.
        """
        # Passa os dados pelo modelo
        output = self.modelo(x)

        # Restringe os valores gerados para estar dentro dos limites observados em cada coluna
        output = (output + 1) * 0.5  # Mapeia para o intervalo (0, 1). Conforme está o DF normalizado
        output = output * (self.max_values - self.min_values) + self.min_values
        
        # Ajustar os valores da coluna específica para os valores mais próximos na lista fornecida
        output[:, self.coluna_indices] = self._ajustar_para_valores_possiveis(output[:, self.coluna_indices])
        
        return output

    def _ajustar_para_valores_possiveis(self, coluna_valores):
        """
        Ajusta os valores da coluna para os valores mais próximos na lista de valores possíveis.

        Parâmetros:
            - coluna_valores (torch.Tensor): O tensor contendo os valores da coluna a serem ajustados.

        Retorna:
            - torch.Tensor: O tensor ajustado com os valores mais próximos na lista de valores possíveis.
        """
        distancias = torch.abs(coluna_valores.unsqueeze(1) - self.valores_possiveis.unsqueeze(0))
        indices_mais_proximos = torch.argmin(distancias, dim=1)
        valores_ajustados = self.valores_possiveis[indices_mais_proximos]
        return valores_ajustados


##### Discriminador

In [125]:
class Discriminador(nn.Module):
    """
    Esta classe recebe um vetor de entrada (representando um registro) e produz uma única saída, 
    que representa a probabilidade do registro ser real.

    Parâmetros:
        - tamanho_entrada (int): O tamanho do vetor de entrada, que representa a dimensão do registro.
    """

    def __init__(self, tamanho_entrada):
        super(Discriminador, self).__init__()

        # Definição da arquitetura do modelo
        self.modelo = nn.Sequential(
            nn.Linear(tamanho_entrada, 128), # Primeira camada de entrada (linear)
            nn.ReLU(), # Capacitar não-linearidade
            nn.Linear(128, 1), 
            nn.Sigmoid() # Saídas em (0 e 1)
        )

    def forward(self, x):
        """
        Essa função passar os dados de entrada através do modelo. Recebe o tensor de entrada "x (torch.Tensor)" e retorna
        o tensor de saída que da a probabilidade do registro ser real. 
        
        Parâmetros:
            - x (torch.Tensor): O tensor de entrada.
        """
        return self.modelo(x)

##### Treinando o discriminador

In [126]:
def treinar_discriminador(discriminador, otimizador, dados_reais, dados_falsos):
    otimizador.zero_grad() # Zera os gradientes
    previsao_real = discriminador(dados_reais) # Passando as 7 instâncias originais pro discriminador
    perda_real = criterio(previsao_real, torch.ones_like(previsao_real)) # Próximo de 1 = verdadeiro, próximo de 0 = falso
    perda_real.backward() # Algoritmo de backpropagation (Feed-Forward e Feed-Backward)

    previsao_falsa = discriminador(dados_falsos)
    perda_falsa = criterio(previsao_falsa, torch.zeros_like(previsao_falsa)) # Próximo de 0 = verdadeiro, próximo de 1 = falso
    perda_falsa.backward()

    otimizador.step() # Atualiza os gradientes

    return perda_real + perda_falsa

##### Treinando o gerador

In [127]:
def treinar_gerador(gerador, otimizador, dados_falsos): # Etapa de duelo entre o gerador e o discriminador
    otimizador.zero_grad()
    previsao = discriminador(dados_falsos) 
    perda = criterio(previsao, torch.ones_like(previsao))
    perda.backward()
    otimizador.step()
    return perda

##### Função de normalização

In [128]:
def normalizar(df): # Garante que os dados estejam no intervalo de (0 e 1)
    return (df - df.min()) / (df.max() - df.min())

##### Função de desnormalização

In [129]:
def desnormalizar(df, df_normalizado): # Retorna os valores ao original
    return df_normalizado * (df.max() - df.min()) + df.min()

##### Normalizando os dados

In [130]:
df_normalizado = normalizar(df)

##### Definindo dimensões

In [131]:
tamanho_entrada = len(df.columns)  # Aplica em todas as colunas da base de dados (24)
tamanho_saida = tamanho_entrada

##### Ajuste de hiperparâmetros

In [132]:
numero_epocas = 5000 # Quantidade de iterações em cima da base
tamanho_batch = 32 # Dados divididos em lotes

taxa_aprendizado = 0.0001
quantidade_novos_registros = 10000

In [133]:
def calcular_min_max(df):
    """
    Calcula os valores mínimos e máximos de cada coluna do dataframe.

    Parâmetros:
        - df (pandas.DataFrame): O dataframe contendo os dados.

    Retorna:
        - min_values (torch.Tensor): O tensor contendo os valores mínimos para cada coluna.
        - max_values (torch.Tensor): O tensor contendo os valores máximos para cada coluna.
    """
    min_values = torch.tensor(df.min().values, dtype=torch.float32)
    max_values = torch.tensor(df.max().values, dtype=torch.float32)
    return min_values, max_values

min_values, max_values = calcular_min_max(df_normalizado)

In [135]:
# Iniciando o gerador e o discriminador
gerador = Gerador(tamanho_entrada, tamanho_saida, min_values, max_values, 4, [0, 1])
discriminador = Discriminador(tamanho_entrada)

In [136]:
# Definindo a função de perda
criterio = nn.BCELoss() # Entropia cruzada para casos binários

# Definindo os otimizadores
otimizador_gerador = optim.Adam(gerador.parameters(), lr=taxa_aprendizado) # Estava usando antes o (SGD)
otimizador_discriminador = optim.Adam(discriminador.parameters(), lr=taxa_aprendizado)

##### Treinando a GAN

In [ ]:
torch.autograd.set_detect_anomaly(True)

In [ ]:
for epoca in range(numero_epocas):
    for i in range(0, len(df_normalizado), tamanho_batch):
        # Colocando os dados reais no formato que o PyTorch interpreta
        dados_reais = torch.tensor(df_normalizado.iloc[i:i+tamanho_batch].values, dtype=torch.float32)
        
        # Dados falsos feitos pelo Gerador
        ruido = torch.randn(tamanho_batch, tamanho_entrada)
        dados_falsos = gerador(ruido)

        # Treinando o Discriminador
        perda_discriminador = treinar_discriminador(discriminador, otimizador_discriminador, dados_reais, dados_falsos)

        # Treinando o Gerador
        ruido = torch.randn(tamanho_batch, tamanho_entrada)
        dados_falsos = gerador(ruido)
        perda_gerador = treinar_gerador(gerador, otimizador_gerador, dados_falsos)

    # Imprimir progresso a cada 100 épocas
    if (epoca+1) % 100 == 0:
        print(f"Época [{epoca+1}/{numero_epocas}], Perda do Discriminador: {perda_discriminador.item()}, Perda do Gerador: {perda_gerador.item()}")

##### Gerando dados sintéticos e aplicando restrições

In [139]:
# Gerar dados sintéticos para toda a base de dados
novos_registros = []
for _ in range(quantidade_novos_registros // tamanho_batch):
    ruido = torch.randn(tamanho_batch, tamanho_entrada)
    batch_sintetico = gerador(ruido).detach().numpy()

    novos_registros.append(batch_sintetico)


# Quando o número de dados gerados não é múltiplo do tamanho do batch
dados_restantes = quantidade_novos_registros % tamanho_batch
if dados_restantes > 0:
    ruido = torch.randn(dados_restantes, tamanho_entrada)
    batch_sintetico = gerador(ruido).detach().numpy()
    
    # Restrição 1: Valores negativos iguais a zero
    batch_sintetico[batch_sintetico < 0] = 0

    novos_registros.append(batch_sintetico)

##### Desnormalizando e salvando

In [140]:
novos_registros = np.concatenate(novos_registros)
novos_registros = desnormalizar(df, pd.DataFrame(novos_registros, columns=df.columns))

In [141]:
novos_registros.to_csv('../registros_gan.csv', index=False)

##### Função para transformar os dados e padronizar conforme o original

In [150]:
def transforma_int(coluna):
    return int(coluna)

In [151]:
def transforma_float(column):
    return round(float(column), 3)

In [152]:
def mapear_0_ou_1(valor):
    if valor <= 0.5:
        return 0
    else:
        return 1

In [153]:
def mapear_type_of_production(valor):
    if valor <= 1.5:
        return 1
    elif valor <= 2.5:
        return 2
    else:
        return 3

##### Separando as colunas para os tipos de transformação

In [ ]:
df_gan.columns

In [155]:
colunas_float = ['height_of_jacket_or_sub-structure (m)', 'height_of_jacket_or_sub-structure (m)', 'risk_to_personnel-complete', 'risk_to_personnel-partial', 'technical_feasibility_or_challenge-complete', 'technical_feasibility_or_challenge-partial', 'commercial_impact_on_fisheries-complete', 'commercial_impact_on_fisheries-partial', 'wider_community_impact-complete', 'wider_community_impact-partial', 'total_removal_cost-complete', 'total_removal_cost-partial', 'impacts_of_option-complete']
colunas_int = ['water_depth (m)', 'installation_date', 'weight (t)', 'number_of_legs', 'number_of_piles', 'distance_to_coast (km)', 'energy_consumption-complete (GJ)', 'energy_consumption-partial (GJ)', 'emissions-complete (t)', 'emissions-partial (t)']
coluna_mapear_0_ou_1 = ['risk_to_other_users-complete', 'impacts_of_option-partial']

##### Aplicando as transformações

In [156]:

df_gan['type_of_production (1 oil and gas; 2 oil; 3 gas)'] = df_gan['type_of_production (1 oil and gas; 2 oil; 3 gas)'].apply(mapear_type_of_production)

for coluna in colunas_int:
    df_gan[coluna] = df_gan[coluna].apply(transforma_int)


#### Salvando resultados

In [158]:
df_gan.to_csv('../dados/registros_gan.csv', index=False)

In [ ]:
import pandas as pd
from sdmetrics.reports.single_table import QualityReport
from sdv.metadata import SingleTableMetadata

# Carregar os dados originais e sintéticos
df_original = pd.read_csv('../imputed_plataformas_otc.csv')  # Seu dataset original
df_sintetico = pd.read_csv('../dados/registros_gan.csv')  # Dados sintéticos gerados

metadata_dict = {
    "columns": {
        "water_depth (m)": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "weight (t)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "numerical"
        },
        "installation_date": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "type_of_production (1 oil and gas; 2 oil; 3 gas)": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "number_of_legs": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "number_of_piles": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "height_of_jacket_or_sub-structure (m)": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "distance_to_coast (km)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "numerical"
        },
        "energy_consumption-complete (GJ)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "numerical"
        },
        "energy_consumption-partial (GJ)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "numerical"
        },
        "emissions-complete (t)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "numerical"
        },
        "emissions-partial (t)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "numerical"
        },
        "recommended (1 partial; 2 complete)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "numerical"
        }
    },
    "primary_key": "water_depth (m)",
    "METADATA_SPEC_VERSION": "SINGLE_TABLE_V1"
}

# Gerar o relatório de qualidade entre os dados reais e sintéticos
report = QualityReport()
report.generate(real_data=df_original, synthetic_data=df_sintetico, metadata=metadata_dict)

# Exibir detalhes específicos do relatório
print(report.get_details(property_name='Column Shapes'))
print(report.get_details(property_name='Column Pair Trends'))
